In [1]:
import duckdb
import pandas as pd
import polars as pl

In [2]:
csv_names_dir = 'data/input/names.csv'

# Leitura de arquivo .csv

In [3]:
df_pandas_names = pd.read_csv(csv_names_dir)

In [4]:
df_polars_names = pl.read_csv(csv_names_dir)

In [5]:
duckdb.sql(f""" DROP TABLE IF EXISTS df_duckdb_names;
 """)

duckdb.sql(f"""
                CREATE TABLE df_duckdb_names AS 
                SELECT * FROM '{csv_names_dir}'
""")

# Replace valores \N

Pandas

In [6]:
for column in df_pandas_names.columns:
    print(f'{column}: ', df_pandas_names.query(f"{column} == '\\\\N'")[column].count())

nconst:  0
primaryName:  48
birthYear:  12770823
deathYear:  13166298
primaryProfession:  2565021
knownForTitles:  1470866


In [7]:
df_pandas_names = df_pandas_names.replace('\\\\N', None, regex=True)

Polars

In [8]:
for column in df_polars_names.columns:
    print(f'{column}: ', df_polars_names.select(pl.col(column)).filter(pl.col(column) == '\\N').count())

nconst:  shape: (1, 1)
┌────────┐
│ nconst │
│ ---    │
│ u32    │
╞════════╡
│ 0      │
└────────┘
primaryName:  shape: (1, 1)
┌─────────────┐
│ primaryName │
│ ---         │
│ u32         │
╞═════════════╡
│ 48          │
└─────────────┘
birthYear:  shape: (1, 1)
┌───────────┐
│ birthYear │
│ ---       │
│ u32       │
╞═══════════╡
│ 12770823  │
└───────────┘
deathYear:  shape: (1, 1)
┌───────────┐
│ deathYear │
│ ---       │
│ u32       │
╞═══════════╡
│ 13166298  │
└───────────┘
primaryProfession:  shape: (1, 1)
┌───────────────────┐
│ primaryProfession │
│ ---               │
│ u32               │
╞═══════════════════╡
│ 2565021           │
└───────────────────┘
knownForTitles:  shape: (1, 1)
┌────────────────┐
│ knownForTitles │
│ ---            │
│ u32            │
╞════════════════╡
│ 1470866        │
└────────────────┘


In [9]:
df_polars_names = df_polars_names.with_columns(pl.all().replace('\\N', None))

DuckDB

In [10]:
duckdb.sql("""
    SELECT 
            SUM(CASE WHEN nconst             == '\\N' THEN 1 ELSE 0 END) AS nconst
           ,SUM(CASE WHEN primaryName        == '\\N' THEN 1 ELSE 0 END) AS primaryName
           ,SUM(CASE WHEN birthYear          == '\\N' THEN 1 ELSE 0 END) AS birthYear
           ,SUM(CASE WHEN deathYear          == '\\N' THEN 1 ELSE 0 END) AS deathYear
           ,SUM(CASE WHEN primaryProfession  == '\\N' THEN 1 ELSE 0 END) AS primaryProfession
           ,SUM(CASE WHEN knownForTitles     == '\\N' THEN 1 ELSE 0 END) AS knownForTitles
    FROM df_duckdb_names 
""").show()

┌────────┬─────────────┬───────────┬───────────┬───────────────────┬────────────────┐
│ nconst │ primaryName │ birthYear │ deathYear │ primaryProfession │ knownForTitles │
│ int128 │   int128    │  int128   │  int128   │      int128       │     int128     │
├────────┼─────────────┼───────────┼───────────┼───────────────────┼────────────────┤
│      0 │          48 │  12770823 │  13166298 │           2565021 │        1470866 │
└────────┴─────────────┴───────────┴───────────┴───────────────────┴────────────────┘



In [11]:
duckdb.sql("""
    UPDATE df_duckdb_names
    SET 
           nconst               = CASE WHEN nconst              == '\\N' THEN NULL ELSE nconst END,
           primaryName          = CASE WHEN primaryName         == '\\N' THEN NULL ELSE primaryName END,
           birthYear            = CASE WHEN birthYear           == '\\N' THEN NULL ELSE birthYear END,
           deathYear            = CASE WHEN deathYear           == '\\N' THEN NULL ELSE deathYear END,
           primaryProfession    = CASE WHEN primaryProfession   == '\\N' THEN NULL ELSE primaryProfession END,
           knownForTitles       = CASE WHEN knownForTitles      == '\\N' THEN NULL ELSE knownForTitles END;
""")

# Criando tabela profession

Pandas

In [87]:
df_pandas_profession = df_pandas_names[['nconst','primaryProfession']]

In [88]:
profession = df_pandas_profession['primaryProfession'].str.split(',', expand=True)

df_pandas_profession['profession1'] = profession[0]
df_pandas_profession['profession2'] = profession[1]
df_pandas_profession['profession3'] = profession[2]

df_pandas_profession.drop(columns=['primaryProfession'], inplace=True)

C:\Users\danic\AppData\Local\Temp\ipykernel_6852\826471510.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pandas_profession['profession1'] = profession[0]
C:\Users\danic\AppData\Local\Temp\ipykernel_6852\826471510.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pandas_profession['profession2'] = profession[1]
C:\Users\danic\AppData\Local\Temp\ipykernel_6852\826471510.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

Polars

In [14]:
df_polars_profession = df_polars_names.select(['nconst','primaryProfession']).with_columns(pl.col('primaryProfession').str.split(','))

In [15]:
df_polars_profession = df_polars_profession.with_columns(
    pl.col('primaryProfession').list.get(0).alias('profession1'),
    pl.col('primaryProfession').list.get(1).alias('profession2'),
    pl.col('primaryProfession').list.get(2).alias('profession3'),
).select(['nconst','profession1','profession2','profession3'])

DuckDB

In [16]:
duckdb.sql("""
    CREATE TABLE IF NOT EXISTS df_duckdb_profession AS
    SELECT 
           nconst,
           SPLIT_PART(primaryProfession, ',', 1) AS profession1,
           SPLIT_PART(primaryProfession, ',', 2) AS profession2,
           SPLIT_PART(primaryProfession, ',', 3) AS profession3
    FROM df_duckdb_names
""")

# Join

Pandas

In [30]:
df_pandas_names.merge(df_pandas_profession, on='nconst')[['nconst','profession1','profession2','profession3']]

,nconst,profession1,profession2,profession3
0,nm0000001,actor,miscellaneous,producer
1,nm0000002,actress,soundtrack,archive_footage
2,nm0000003,actress,music_department,producer
3,nm0000004,actor,writer,music_department
4,nm0000005,writer,director,actor
...,...,...,...,...
13393223,nm9993714,animation_department,art_department,None
13393224,nm9993716,None,None,None
13393225,nm9993717,cinematographer,None,None
13393226,nm9993718,cinematographer,None,None


Polars

In [28]:
df_polars_names.join(df_polars_profession, on='nconst').select(['nconst','profession1','profession2','profession3'])

nconst,profession1,profession2,profession3
str,str,str,str
"""nm0000001""","""actor""","""miscellaneous""","""producer"""
"""nm0000002""","""actress""","""soundtrack""","""archive_footage"""
"""nm0000003""","""actress""","""music_department""","""producer"""
"""nm0000004""","""actor""","""writer""","""music_department"""
"""nm0000005""","""writer""","""director""","""actor"""
…,…,…,…
"""nm9993714""","""animation_department""","""art_department""",null
"""nm9993716""",null,null,null
"""nm9993717""","""cinematographer""",null,null


DuckDB

In [19]:
duckdb.sql("""
    SELECT 
        n.nconst,
        p.profession1,
        p.profession2,
        p.profession3
    FROM df_duckdb_names n
    JOIN df_duckdb_profession p
    ON n.nconst = p.nconst
""").show()

┌───────────┬───────────────────────────┬─────────────────┬──────────────────────┐
│  nconst   │        profession1        │   profession2   │     profession3      │
│  varchar  │          varchar          │     varchar     │       varchar        │
├───────────┼───────────────────────────┼─────────────────┼──────────────────────┤
│ nm0654525 │ actor                     │                 │                      │
│ nm0654526 │ miscellaneous             │ writer          │ actor                │
│ nm0654528 │ actor                     │                 │                      │
│ nm0654530 │ actor                     │ archive_footage │                      │
│ nm0654531 │ actor                     │ writer          │ producer             │
│ nm0654532 │ director                  │ writer          │ assistant_director   │
│ nm0654534 │ actor                     │                 │                      │
│ nm0654535 │ director                  │ producer        │ writer               │
│ nm

# Funções agregadas

Pandas

In [89]:
df_pandas_profession[['profession1']].groupby('profession1').size().reset_index(name='count').sort_values(by='count', ascending=False)

,profession1,count
1,actor,2858675
2,actress,1791469
27,miscellaneous,946561
31,producer,701190
10,camera_department,595239
18,director,419889
45,writer,384036
6,art_department,354949
38,sound_department,279346
15,composer,244672


Polars

In [59]:
df_polars_profession.select('profession1').group_by('profession1').len(name='count').sort('count', descending=True)

profession1,count
str,u32
"""actor""",2858675
null,2565021
"""actress""",1791469
"""miscellaneous""",946561
"""producer""",701190
…,…
"""podcaster""",264
"""production_department""",104
"""choreographer""",73


DuckDB

In [79]:
duckdb.sql("""
    SELECT 
        profession1,
        count(1) as count
    FROM df_duckdb_profession 
    GROUP BY 1
    ORDER BY 2 DESC
""").show()

┌───────────────────────┬─────────┐
│      profession1      │  count  │
│        varchar        │  int64  │
├───────────────────────┼─────────┤
│ actor                 │ 2858675 │
│                       │ 2565021 │
│ actress               │ 1791469 │
│ miscellaneous         │  946561 │
│ producer              │  701190 │
│ camera_department     │  595239 │
│ director              │  419889 │
│ writer                │  384036 │
│ art_department        │  354949 │
│ sound_department      │  279346 │
│      ·                │      ·  │
│      ·                │      ·  │
│      ·                │      ·  │
│ music_artist          │    3819 │
│ publicist             │    2869 │
│ legal                 │    2722 │
│ assistant             │    1816 │
│ archive_sound         │     984 │
│ podcaster             │     264 │
│ production_department │     104 │
│ choreographer         │      73 │
│ accountant            │      15 │
│ electrical_department │      13 │
├───────────────────────┴───

# Amostra

In [17]:
df_pandas_names.head(5)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,None,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"


In [18]:
df_pandas_profession.head(5)

,nconst,profession1,profession2,profession3
0,nm0000001,actor,miscellaneous,producer
1,nm0000002,actress,soundtrack,archive_footage
2,nm0000003,actress,music_department,producer
3,nm0000004,actor,writer,music_department
4,nm0000005,writer,director,actor


In [19]:
df_polars_names.head(5)

nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
str,str,str,str,str,str
"""nm0000001""","""Fred Astaire""","""1899""","""1987""","""actor,miscellaneous,producer""","""tt0072308,tt0050419,tt0053137,…"
"""nm0000002""","""Lauren Bacall""","""1924""","""2014""","""actress,soundtrack,archive_foo…","""tt0037382,tt0075213,tt0117057,…"
"""nm0000003""","""Brigitte Bardot""","""1934""",null,"""actress,music_department,produ…","""tt0057345,tt0049189,tt0056404,…"
"""nm0000004""","""John Belushi""","""1949""","""1982""","""actor,writer,music_department""","""tt0072562,tt0077975,tt0080455,…"
"""nm0000005""","""Ingmar Bergman""","""1918""","""2007""","""writer,director,actor""","""tt0050986,tt0083922,tt0050976,…"


In [20]:
df_polars_profession.head(5)

nconst,profession1,profession2,profession3
str,str,str,str
"""nm0000001""","""actor""","""miscellaneous""","""producer"""
"""nm0000002""","""actress""","""soundtrack""","""archive_footage"""
"""nm0000003""","""actress""","""music_department""","""producer"""
"""nm0000004""","""actor""","""writer""","""music_department"""
"""nm0000005""","""writer""","""director""","""actor"""


In [21]:
duckdb.sql("""
    SELECT *
    FROM df_duckdb_names
    LIMIT 5
""").show() 

┌───────────┬─────────────────┬───────────┬───────────┬──────────────────────┬─────────────────────────────────────────┐
│  nconst   │   primaryName   │ birthYear │ deathYear │  primaryProfession   │             knownForTitles              │
│  varchar  │     varchar     │  varchar  │  varchar  │       varchar        │                 varchar                 │
├───────────┼─────────────────┼───────────┼───────────┼──────────────────────┼─────────────────────────────────────────┤
│ nm0000001 │ Fred Astaire    │ 1899      │ 1987      │ actor,miscellaneou…  │ tt0072308,tt0050419,tt0053137,tt0027125 │
│ nm0000002 │ Lauren Bacall   │ 1924      │ 2014      │ actress,soundtrack…  │ tt0037382,tt0075213,tt0117057,tt0038355 │
│ nm0000003 │ Brigitte Bardot │ 1934      │ NULL      │ actress,music_depa…  │ tt0057345,tt0049189,tt0056404,tt0054452 │
│ nm0000004 │ John Belushi    │ 1949      │ 1982      │ actor,writer,music…  │ tt0072562,tt0077975,tt0080455,tt0078723 │
│ nm0000005 │ Ingmar Bergman  │ 

In [22]:
duckdb.sql("""
    SELECT *
    FROM df_duckdb_profession
    LIMIT 5
""").show() 

┌───────────┬─────────────┬──────────────────┬──────────────────┐
│  nconst   │ profession1 │   profession2    │   profession3    │
│  varchar  │   varchar   │     varchar      │     varchar      │
├───────────┼─────────────┼──────────────────┼──────────────────┤
│ nm0000001 │ actor       │ miscellaneous    │ producer         │
│ nm0000002 │ actress     │ soundtrack       │ archive_footage  │
│ nm0000003 │ actress     │ music_department │ producer         │
│ nm0000004 │ actor       │ writer           │ music_department │
│ nm0000005 │ writer      │ director         │ actor            │
└───────────┴─────────────┴──────────────────┴──────────────────┘

